# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute_target import ComputeTargetException

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'AutoMLversion'

experiment=Experiment(ws, experiment_name)

In [ ]:
amlcompute_cluster_name = "automl-compute"

# Verify that cluster does not exist already
try:
    training_cluster = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=6)
    training_cluster = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

training_cluster.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)


In [ ]:
found = False
key = "**Gender Classification**"
description_text = "**Gender Classification Dataset for Capstone Peoject Udacity**"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/vagbharathi/Capstone-project-UdacityMachineLearningEngineer-MicrosoftAzure-Scholarship/main/nd00333-capstone-master/starter_file/gender-classification.csv
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
dataset.take(5).to_pandas_dataframe()

In [ ]:
# Split data into train and test sets
train, test = dataset.random_split(percentage=0.75, seed=121)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 120,
    "max_concurrent_iterations": 6,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=training_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             n_cross_validations=5,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

In [ ]:
#TODO: Save the best model
from azureml.core.model import Model
best_run.register_model(model_path='outputs/model.pkl', model_name='heartautoml_model', model_framework=Model.Framework.SCIKITLEARN)
model = ws.models['heartautoml_model']

In [ ]:
print(model)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core import Environment
best_run.download_file('outputs/conda_env_v_1_0_0.yml','conda_env.yml')
myenv = Environment.from_conda_specification(name="myenv", file_path="conda_env.yml")
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(entry_script='score.py',
                                   environment=myenv)

service_name = 'automl-service'
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled=True)
aci_service = Model.deploy(workspace=ws,
                           name= service_name,
                           models= [model],
                           inference_config= inference_config,
                           deployment_config=deployment_config)
aci_service.wait_for_deployment(show_output = True)

In [ ]:
print(aci_service.state)

In [ ]:
# Enable AppInsights
aci_service.update(enable_app_insights=True)
print('AppInsights enabled!')

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
endpoint = aci_service.scoring_uri
primary, secondary = aci_service.get_keys()
print(primary)
print(endpoint)

In [ ]:
!python endpoint.py

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
aci_service.delete()
print('Service deleted.')